### Part 2: ML pipeline

In [1]:
import numpy as np
import pandas as pd
import re

from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv('../input_data/news_paper_data.csv')

In [3]:
df.head()

,node_id,title,date,url,category,article
0,3074331,Consumer rights body’s oil price monitoring st...,19-07-2022,/business/news/consumer-rights-bodys-oil-price...,business,The Directorate of National Consumer Rights Pr...
1,3074126,Chattogram shares bleed for another day,19-07-2022,/business/news/chattogram-shares-bleed-another...,business,Shares on the Chittagong Stock Exchange (CSE) ...
2,3073706,bKash’s instant add money service at Dev Bank,19-07-2022,/business/organisation-news/news/bkashs-instan...,business,Clients of Bangladesh Development Bank (BDB) c...
3,3074291,Textile millers demand 360 days for deferred L...,19-07-2022,/business/news/textile-millers-demand-360-days...,business,The leaders of Bangladesh Textile Mills Associ...
4,3074276,"Tk 25,000 crore refinance scheme unveiled for ...",19-07-2022,/business/news/tk-25000-crore-refinance-scheme...,business,"Bangladesh Bank today unveiled a Tk 25,000 cro..."


In [4]:
# Data Cleaning 
def cleaning_text(text):
    text = text.lower().replace('\n', ' ').replace('\r', ' ').strip()
    
    # remove extra spaces
    text = re.sub(' +', ' ', text)
    
    # only took words
    text = re.sub(r'[^\w\s]', '', text)
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    
    clean_text = [w for w in word_tokens if w not in stop_words]
    
    return ' '.join(clean_text)

In [5]:
df['clean_data'] = df['article'].apply(cleaning_text)

df.head()

,node_id,title,date,url,category,article,clean_data
0,3074331,Consumer rights body’s oil price monitoring st...,19-07-2022,/business/news/consumer-rights-bodys-oil-price...,business,The Directorate of National Consumer Rights Pr...,directorate national consumer rights protectio...
1,3074126,Chattogram shares bleed for another day,19-07-2022,/business/news/chattogram-shares-bleed-another...,business,Shares on the Chittagong Stock Exchange (CSE) ...,shares chittagong stock exchange cse witnessed...
2,3073706,bKash’s instant add money service at Dev Bank,19-07-2022,/business/organisation-news/news/bkashs-instan...,business,Clients of Bangladesh Development Bank (BDB) c...,clients bangladesh development bank bdb use ad...
3,3074291,Textile millers demand 360 days for deferred L...,19-07-2022,/business/news/textile-millers-demand-360-days...,business,The leaders of Bangladesh Textile Mills Associ...,leaders bangladesh textile mills association b...
4,3074276,"Tk 25,000 crore refinance scheme unveiled for ...",19-07-2022,/business/news/tk-25000-crore-refinance-scheme...,business,"Bangladesh Bank today unveiled a Tk 25,000 cro...",bangladesh bank today unveiled tk 25000 crore ...


In [6]:
label_encoder = preprocessing.LabelEncoder()
df['category_label'] = label_encoder.fit_transform(df['category'])

In [7]:
df.head()

,node_id,title,date,url,category,article,clean_data,category_label
0,3074331,Consumer rights body’s oil price monitoring st...,19-07-2022,/business/news/consumer-rights-bodys-oil-price...,business,The Directorate of National Consumer Rights Pr...,directorate national consumer rights protectio...,0
1,3074126,Chattogram shares bleed for another day,19-07-2022,/business/news/chattogram-shares-bleed-another...,business,Shares on the Chittagong Stock Exchange (CSE) ...,shares chittagong stock exchange cse witnessed...,0
2,3073706,bKash’s instant add money service at Dev Bank,19-07-2022,/business/organisation-news/news/bkashs-instan...,business,Clients of Bangladesh Development Bank (BDB) c...,clients bangladesh development bank bdb use ad...,0
3,3074291,Textile millers demand 360 days for deferred L...,19-07-2022,/business/news/textile-millers-demand-360-days...,business,The leaders of Bangladesh Textile Mills Associ...,leaders bangladesh textile mills association b...,0
4,3074276,"Tk 25,000 crore refinance scheme unveiled for ...",19-07-2022,/business/news/tk-25000-crore-refinance-scheme...,business,"Bangladesh Bank today unveiled a Tk 25,000 cro...",bangladesh bank today unveiled tk 25000 crore ...,0


In [8]:
# test train split
X_train, X_test, Y_train, Y_test = train_test_split(df['clean_data'],
                                                   df['category_label'], 
                                                   test_size = 0.2,
                                                   random_state = 8)

In [9]:
print(f'Shape of X: {X_train.shape} and {X_test.shape}')
print(f'Shape of Y: {Y_train.shape} and {Y_test.shape}')

Shape of X: (196,) and (49,)
Shape of Y: (196,) and (49,)


In [12]:
ngram_range = (1, 2)
min_df = 10
max_df = 1.
max_features = 32

tfidf = TfidfVectorizer(encoding='utf-8',
                         ngram_range = ngram_range,
                         min_df = min_df, 
                         max_df = max_df,
                         norm = 'l2',
                         max_features = max_features,
                         stop_words = None,
                         lowercase = False,
                         sublinear_tf = True)

feature_train = tfidf.fit_transform(X_train).toarray()
label_train = Y_train

feature_test = tfidf.fit_transform(X_test).toarray()
label_test = Y_test

print(f'Size of features object: {feature_train.shape} and {feature_test.shape}')

Size of features object: (196, 32) and (49, 32)


In [13]:
# Make random forest classifier
model = RandomForestClassifier()

# fit the model
model.fit(feature_train, label_train)

# get prediction
model_prediction = model.predict(feature_test)

# calculate accuracy
accuracy_val = accuracy_score(label_test, model_prediction)

print(f'Accuracy of the algorithm: {accuracy_val*100} %')

Accuracy of the algorithm: 36.734693877551024 %


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
summary = classification_report(label_test, model_prediction)
print(f'Summary of the model prediction: \n{summary}')

Summary of the model prediction: 
              precision    recall  f1-score   support

           0       0.29      0.11      0.15        19
           1       0.67      0.27      0.38        15
           2       0.33      0.80      0.47        15

   micro avg       0.37      0.37      0.37        49
   macro avg       0.43      0.39      0.34        49
weighted avg       0.42      0.37      0.32        49

